In [12]:
pip install ssgetpy scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [14]:
# In Jupyter Notebook, prefix with an exclamation mark:
!pip install --user ssgetpy scipy networkx


[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [22]:
import ssgetpy
import gzip
import shutil
import os
from scipy.io import mmread
from scipy.sparse.csgraph import reverse_cuthill_mckee
import numpy as np


In [23]:
# 1. Fetch the 1138_bus matrix and get the path to its downloaded file
mat = ssgetpy.fetch(1138)[0]
downloaded_paths = mat.download()       # e.g., ['/home/.../1138_bus.mtx.gz']
gz_path = downloaded_paths[0]

# 2. Derive mtx_path by stripping exactly one “.gz” (if present)
base, ext = os.path.splitext(gz_path)
if ext.lower() == '.gz':
    mtx_path = base                # now ends with “.mtx”
else:
    mtx_path = gz_path             # already uncompressed

In [24]:
if ext.lower() == '.gz':
    with gzip.open(gz_path, 'rb') as f_in, open(mtx_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# 4. Verify the banner (optional)
with open(mtx_path, 'rt') as f:
    print("Banner:", f.readline().strip())

Banner: oscil_dcop_27/oscil_dcop_27.mtx                                                                     100600       0       0        75021 10557526520  15066  0                                                                                                    ustar   davis                                                                0       0                                                                                                                                                                         %%MatrixMarket matrix coordinate real general


In [26]:
A = mmread(gz_path).tocsr()

ValueError: Line 1: Not a Matrix Market file. Missing banner.

In [27]:
import gzip
from scipy.io import mmread
from scipy.sparse.csgraph import reverse_cuthill_mckee
import ssgetpy
import numpy as np

# 1. Download the 1138_bus matrix (gzipped .mtx.gz)
mat = ssgetpy.fetch(1138)[0]
gz_path = mat.download()[0]

# 2. Open gzip in text mode and pass to mmread
with gzip.open(gz_path, 'rt') as f:
    A = mmread(f).tocsr()

# 3. Reorder and compute bandwidth
perm = reverse_cuthill_mckee(A, symmetric_mode=True)
A_rcm = A[perm, :][:, perm]
rows, cols = A_rcm.nonzero()
bandwidth = np.abs(rows - cols).max()
print("Bandwidth:", bandwidth)  # ≈132


ValueError: Line 1: Not a Matrix Market file. Missing banner.

In [28]:
pip install fast_matrix_market



   ---------------------------------------- 0.0/579.8 kB ? eta -:--:--
   -- ------------------------------------- 30.7/579.8 kB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 81.9/579.8 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 143.4/579.8 kB 1.1 MB/s eta 0:00:01
   ------------- -------------------------- 194.6/579.8 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 245.8/579.8 kB 1.1 MB/s eta 0:00:01
   --------------------- ------------------ 307.2/579.8 kB 1.1 MB/s eta 0:00:01
   ------------------------- -------------- 368.6/579.8 kB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 440.3/579.8 kB 1.2 MB/s eta 0:00:01
   ---------------------------------- ----- 501.8/579.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 579.8/579.8 kB 1.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [34]:
from networkx.convert_matrix import from_scipy_sparse_matrix
G = from_scipy_sparse_matrix(A)


ImportError: cannot import name 'from_scipy_sparse_matrix' from 'networkx.convert_matrix' (C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\networkx\convert_matrix.py)

In [35]:
pip install --upgrade networkx



[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: C:\Users\ASUS\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [36]:
import gzip
from scipy.io import mmread
from scipy.sparse.csgraph import reverse_cuthill_mckee
import numpy as np
import networkx as nx
from networkx.convert_matrix import from_scipy_sparse_array  # updated import

# … load A as a scipy CSR matrix …

# build graph
G = from_scipy_sparse_array(A)
# … rest of workflow …


NameError: name 'A' is not defined

In [37]:
# Install required packages if not already installed
# In a Jupyter cell, run:
# !pip install ssgetpy scipy

import ssgetpy
import gzip
import shutil
import os

# 1. Fetch and download the 1138_bus matrix (gzipped .mtx.gz)
mat = ssgetpy.fetch(1138)[0]
gz_path = mat.download()[0]
print("Downloaded (gzipped) MatrixMarket file at:", gz_path)

# 2. Derive the target .mtx path by stripping exactly one “.gz” extension
base, ext = os.path.splitext(gz_path)
if ext.lower() == '.gz':
    mtx_path = base             # will end with ".mtx"
else:
    raise RuntimeError(f"Expected a .gz file but got '{gz_path}'")

# 3. Decompress the .mtx.gz file to .mtx
with gzip.open(gz_path, 'rb') as f_in, open(mtx_path, 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)

print("Decompressed MatrixMarket file saved at:", mtx_path)


Downloaded (gzipped) MatrixMarket file at: C:\Users\ASUS\AppData\Roaming\ssgetpy\MM\Sandia\oscil_dcop_27.tar.gz
Decompressed MatrixMarket file saved at: C:\Users\ASUS\AppData\Roaming\ssgetpy\MM\Sandia\oscil_dcop_27.tar


In [38]:
import networkx as nx

def load_mtx_as_graph(path: str, zero_based: bool = True) -> nx.Graph:
    """
    Load a symmetric sparse matrix from Matrix Market (.mtx) into a NetworkX Graph.
    
    Parameters
    ----------
    path
        Path to the .mtx file (plain-text, decompressed).
    zero_based
        If True, treat indices in the file as 1-based and shift them to 0-based.
        Set to False if the .mtx is already 0-based.
        
    Returns
    -------
    G : nx.Graph
        An undirected NetworkX graph whose adjacency corresponds to the matrix.
    """
    G = nx.Graph()
    with open(path, 'rt') as f:
        # Skip comments and read header
        for line in f:
            line = line.strip()
            if line == '' or line.startswith('%'):
                continue
            # First non-comment non-blank line: dimensions
            n_rows, n_cols, n_nonzeros = map(int, line.split())
            break
        
        # Read each entry as an edge
        for lineno, line in enumerate(f, start=1):
            if line.strip() == '' or line.startswith('%'):
                continue
            i_str, j_str, *rest = line.split()
            i, j = int(i_str), int(j_str)
            # Convert to 0-based if needed
            if zero_based:
                i -= 1
                j -= 1
            # For symmetric matrices, only add one edge
            G.add_edge(i, j)
    
    return G

# Usage example:
# Assuming you have already decompressed '1138_bus.mtx.gz' to '1138_bus.mtx'.
graph = load_mtx_as_graph('1138_bus.mtx')
print("Loaded graph with", graph.number_of_nodes(), "nodes and", graph.number_of_edges(), "edges.")


FileNotFoundError: [Errno 2] No such file or directory: '1138_bus.mtx'

In [40]:
import ssgetpy
import os

# 1. Fetch the matrix (returns a MatrixList)
mat_list = ssgetpy.fetch(1138)

# 2. Extract the only Matrix object
mat = mat_list[0]

# 3. Ask it where it downloaded the file(s)
paths = mat.download()  
print("Downloaded file paths:")
for p in paths:
    print("  ", p)

# 4. To see the directory it lives in:
cache_dir = os.path.dirname(paths[0])
print("Cache directory:", cache_dir)


Downloaded file paths:
   C:\Users\ASUS\AppData\Roaming\ssgetpy\MM\Sandia\oscil_dcop_27.tar.gz
   C:\Users\ASUS\AppData\Roaming\ssgetpy\MM\Sandia\oscil_dcop_27.tar.gz
Cache directory: C:\Users\ASUS\AppData\Roaming\ssgetpy\MM\Sandia


In [41]:
# Suppose your mtx_path was obtained from ssgetpy.download(), but contains nulls:
mtx_path_raw = mat.download()[0]

# Strip any trailing null bytes or whitespace:
mtx_path = mtx_path_raw.rstrip('\x00').strip()

# Now pass this clean path to load_mtx_as_graph:
graph = load_mtx_as_graph(mtx_path)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 70: character maps to <undefined>